In [ ]:
#hide 

%load_ext autoreload
%autoreload 2

In [ ]:
#default_exp analysis

# Analysis
> We will test the performance of the model against the test set and visualize some of the results. We will then make the necessary recommendations on how to improve our model further for use cases.

From the results obtained in the training and validation datasets, we did not bother trying to test our model on the test set as the results are likely to be predictably gibberish. We can tell that our model is not learning anything meaningful as of now. Nevertheless, we shall provide an in-depth analysis of what has gone wrong and the next steps we would take to improve results if we had more time at hand. 

## Data

We have attempted to remove stopwords from our taglines and overview to no avail. It is likely that the removal of stopwords did not help as transformer models are trained on entire wikipedia corpuses, and hence likely associate such stopwords with subtle yet distinct meanings that are not captured by more traditional NLP models, and the class imbalance was not reduced as the [PAD] tokens instead became the most populous token in the absence of the stopwords. 

We could try the following to further improve results: 

- Gathering **more data** beyond the 3000 or so examples we used to model this problem. This is justified as we can clearly see the model starting to overfit without any regularization. 

- Using **more augmentations**, especially on the text and images as a means of synthetically 'creating' more data. This will make our model more generalizable to new examples. 

However, our diagnosis is that this **should not be of top priority** as there are likely to be more pressing issues causing more problems.

# Model

The model is likely to be another cause for concern. We can segregate the potential problems into 2 parts. 

- The first is the encoder portion. This may be an issue as the model is currently overfitting and it could be due to the fact that the network is too complex. We could simplify this by starting with training each individual model separately and calculating each network's loss contribution to the overall loss. 

- The second is the decoder portion. As of now, our simple linear layer is unable to produce meaningful outputs likely because it cannot reproduce the **rich textual information** which a tagline is normally required to have. Essentially, using a pretrained decoder type of architecture such as the one shown below is likely to produce results especially with a pretrained word embedding to map the hidden state back into the vocabulary word space. 

- The last bonus suggestion would be to use some variant of a **generator-discriminator** network instead since taglines are inherently creative, and having a discriminator to force our network to learn to output more human-like and creative taglines would be helpful.

![](transformer.jpg)

## Loss function

Presently, the loss function calculates the categorical cross entropy between the predicted and actual taglines. However, this is **unlikely to be meaningful** as many tokens in the vocabulary (of size 30522) will only appear once, making sparse representations common and class imbalances of stop words more likely. Additionally, there is not significance between the positioning of tokens on this tokenizer's vocabulary; for example, the words 'know', 'against' and 'your' are mapped to token ids of 2113, 2114 and 2115 respectively. These id mappings could quite as easily substituted with one another and there will be no significance in their orderings. Hence, it makes little sense to perform the loss comparison directly in this vector space.

To resolve this, we could attempt one of the following actions:

- Modify the loss function to incorporate focal loss to overcome the class imbalance problem. 

- Use an output embedding layer as per the above diagram which will take the actual taglines as input to generate the predicted taglines.

- Use a reverse embedding layer to convert the actual taglines into the hidden dimension embedding space before calculating the categorical entropy with the model's output. 

## Change of problem formulation

Predicting full taglines is grossly difficult given that much of the vocabulary in these taglines are not available in any part of the input data such as overview or even the poster images. Hence, we could simplify the task to instead to generate **relevant key words** which can be used by the movie directors to generate the final tagline by piecing it together themselves. 

Hence, instead of formulating the probem as a sequence generation and prediction task, we can then reformulate the problem instead to be a **multi-label** prediction task as the sequence of words will no longer be a factor. 